# Behavior cloning flow

Create the full flow for training a model for behavior cloning which is completely separated from the other ones. 

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import socket
import pathlib
import yaml
import tqdm
import papermill
from automate import automate_exprun

In [2]:
# host specific directories
hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    raise Exception("Not configured yet")
elif hostname == "szenes.local":
    bc_path = pathlib.Path("~/Documents/Develop/Data/BerryPicker-BC").expanduser()
    exprun_path = pathlib.Path(bc_path, "exprun")
    result_path = pathlib.Path(bc_path, "result")
else:
    exprun_path = pathlib.Path(Config()["experiment_external"]).expanduser()
    result_path = pathlib.Path(Config()["data_external"]).expanduser()

print(f"Path for external experiments: {exprun_path}")
exprun_path.mkdir(exist_ok=True, parents=True)
print(f"Path for external data: {result_path}")
result_path.mkdir(exist_ok=True, parents=True)

Config().set_experiment_path(exprun_path)
Config().set_experiment_data(result_path)

Hostname is szenes.local
Path for external experiments: /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/exprun
Path for external data: /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/result
***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-szenes.yaml
***ExpRun**: Experiment config path changed to /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/exprun
***ExpRun**: Experiment data path changed to /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/result


In [3]:
# Copy the necessary experiments into the external directory
Config().copy_experiment("demonstration")
Config().copy_experiment("sensorprocessing_conv_vae")
Config().copy_experiment("robot_al5d")
Config().copy_experiment("automate")

***ExpRun**: Experiment demonstration copied to /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/exprun/demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/exprun/sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/exprun/robot_al5d
***ExpRun**: Experiment automate copied to /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/exprun/automate


In [ ]:
def generate_sensorprocessing_conv_vae(exprun_path, results_path, latent_size=128, epochs=5):
    """Generate the experiment for the conv-vae sensorprocessing with the right training data and parameters. 
    Returns the experiment and runname
    FIXME there will be parameters etc"""
    experiment = "sensorprocessing_conv_vae"
    runname = "bc_sensorprocessing_conv_vae"
    
    val = {}
    val["latent_size"] = 128
    val["epochs"] = 5
    val["save_period"] = 5
    # setting the training data
    cam = "dev0"
    demo = "touch-apple"
    training_data = []
    training_data.append([demo, "2025_08-07___15_13_18" , cam])
    val["training_data"] = training_data

    # setting the validation data
    validation_data = []
    validation_data.append([demo, "2025_08_07___15_28_02" , cam])
    val["validation_data"] = validation_data

    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), experiment, runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    return [experiment, runname]

In [ ]:
def generate_behaviorcloning(exprun_path, results_path, latent_size=128, epochs=5):
    """Generate the experiment for the behavior cloning with the right training data and parameters. 
    Returns the experiment and runname
    FIXME there will be parameters etc"""
    experiment = "behaviorcloning"
    runname = "bc_sensorprocessing_conv_vae"
    
    val = {}
    # FIXME: here create the parameters for the bc
    val["latent_size"] = latent_size
    # FIXME: set the training data through some data structure passed as parameter....
    # setting the training data
    cam = "dev0"
    demo = "touch-apple"
    training_data = []
    training_data.append([demo, "2025_08-07___15_13_18" , cam])
    val["training_data"] = training_data

    # setting the validation data
    validation_data = []
    validation_data.append([demo, "2025_08_07___15_28_02" , cam])
    val["validation_data"] = validation_data

    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), experiment, runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "BehaviorCloning"
    v["notebook"] = "behavior_cloning/Train-BehaviorCloning.ipynb"
    params = {}
    params["run"] = runname
    params["external_path"] = exprun_path.as_posix()
    params["data_path"] = result_path.as_posix()
    v["params"] = params

    return {"experiment": experiment, "runname": runname, "automation_entry": v}

### Generate a range of exp/runs to be run

In [5]:
expruns = []
expruns.append(generate_sensorprocessing_conv_vae())

### Generate an automation script. 

* FIXME: the notebooks should have support to set the exprun and results directories from the automation exp.


In [ ]:
value = {}
val = []
value["exps_to_run"] = val

creation_style = "exist-ok"

for exprun in expruns:
    # exp = Config().get_experiment(exprun[0], exprun[1])
    # FIXME: add the automation here or generate the automation script
    v = exprun["automation_entry"]
    v["creation_style"] = creation_style
    val.append(v)

path = pathlib.Path(Config().get_experiment_path(), "automate", "flow_bc.yaml")
with open(path, "w") as f:
    yaml.dump(value, f)

### Run the automation script
FIXME: the existing automation does not take into consideration the external directories the right way. I am trying to fix this. 
FIXME: this should be done such that I can also run it from command line

In [7]:
experiment = "automate"
run = "flow_bc"
exp = Config().get_experiment(experiment, run)

#output_path = pathlib.Path(ext_path, "_automation_output")
#output_filename = f"{notebook_path.stem}_{name}_output{ notebook_path.suffix}"
#output = pathlib.Path(output_path, notebook_path.parent, output_filename)
#output.parent.mkdir(exist_ok=True, parents=True)
#print(output)


for item in tqdm.tqdm(exp["exps_to_run"]):
    print(f"***Automating {item['name']}")
    # FIXME set the output in a complicated way to keep everything at the right position
    
    output = pathlib.Path(result_path, "temporary_notebook_output.ipynb")

    notebook = pathlib.Path("..", item["notebook"])
    print(notebook.exists())



    try:
        papermill.execute_notebook(
            notebook,
            output.absolute(),
            cwd=notebook.parent,
            parameters=item["params"]
        )
    except Exception as e:
        print(f"There was an exception {e}")    

***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Data/BerryPicker-BC/exprun/automate/_defaults_automate.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: automate/flow_bc successfully loaded


  0%|          | 0/1 [00:00<?, ?it/s]/Users/lboloni/Documents/Develop/VirtualEnvs/BerryPicker/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***Automating SensorProcessing-ConvVae
True


100%|██████████| 1/1 [00:02<00:00,  2.31s/it]

There was an exception 
---------------------------------------------------------------------------
Exception encountered at "In [7]":
---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
Cell In[7], line 14
     12 print(f'***Train-Conv-VAE***: Running the trainer from scratch for {vae_config["trainer"]["epochs"]} epochs')
     13 exp.start_timer("training")
---> 14 trainer = train(vae_config)
     15 checkpoint_path = pathlib.Path(trainer.checkpoint_dir, f"checkpoint-epoch{trainer.epochs}.pth")
     17 json_path = pathlib.Path(trainer.checkpoint_dir, "config.json")

File ~/Documents/Develop/Github/BerryPicker/BerryPicker/src/sensorprocessing/conv_vae.py:77, in train(config)
     73 logger = logging.getLogger(__name__)
     76 # setup data_loader instances
---> 77 data_loader = config.init_obj('data_loader', module_data)
     78 valid_data_loader = data_loader.split_validation()
     80 